In [1]:
import pathlib
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, Point
import warnings
warnings.filterwarnings(action='ignore')
#import descartes
import folium
from tqdm.notebook import tqdm

In [2]:
%cd E:\yujin\INHA\data

E:\yujin\INHA\data


# 열 순서

category, 좌표값, 인구분포,면적, 주차대수, 행정동 정보,행정동 별 전기차 개수

# 주차장 전처리

#### 02.광양시_주차장_공간정보 + 26.광양시_도시계획(교통시설)

## 1) 02.광양시_주차장_공간정보 --> parkingLot

In [7]:
gyboundary = gpd.read_file('20.광양시_행정경계(읍면동).geojson')
gyboundary

,ADM_DR_CD,ADM_DR_NM,geometry
0,3606031,봉강면,"MULTIPOLYGON (((127.58037 35.10902, 127.58052 ..."
1,3606032,옥룡면,"MULTIPOLYGON (((127.59872 35.12006, 127.59919 ..."
2,3606033,옥곡면,"MULTIPOLYGON (((127.64878 35.03547, 127.64902 ..."
3,3606034,진상면,"MULTIPOLYGON (((127.65639 35.11680, 127.65698 ..."
4,3606035,진월면,"MULTIPOLYGON (((127.75715 35.05502, 127.75716 ..."
5,3606036,다압면,"MULTIPOLYGON (((127.64117 35.16850, 127.64140 ..."
6,3606051,골약동,"MULTIPOLYGON (((127.68056 34.92848, 127.68105 ..."
7,3606053,중마동,"MULTIPOLYGON (((127.70376 34.96198, 127.70542 ..."
8,3606054,광영동,"MULTIPOLYGON (((127.71676 34.96939, 127.71691 ..."
9,3606055,태인동,"MULTIPOLYGON (((127.76162 34.95988, 127.76163 ..."


In [8]:
parkingLot = pd.read_csv('02.광양시_주차장_공간정보.csv')

parkingLot['lon'] = parkingLot['lon'].astype(float)
parkingLot['lat'] = parkingLot['lat'].astype(float)
parkingLot['geometry'] = parkingLot.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)
parkingLot = gpd.GeoDataFrame(parkingLot, geometry='geometry')
parkingLot = parkingLot[['면적','주차면수','geometry']]
parkingLot.rename(columns={"면적":"area"}, inplace=True)

pllist = []
for i in range(len(parkingLot)):
    for j in range(len(gyboundary)):
        if parkingLot.geometry[i].within(gyboundary.geometry[j]) == True:
            pllist.append(gyboundary.ADM_DR_NM[j])

parkingLot['administration'] = pllist
parkingLot

,area,주차면수,geometry,administration
0,4189.0,161.0,POINT (127.58688 34.97032),광양읍
1,2968.0,125.0,POINT (127.69543 34.93625),중마동
2,2836.0,89.0,POINT (127.58459 34.97564),광양읍
3,1980.0,52.0,POINT (127.58068 34.97013),광양읍
4,1970.0,64.0,POINT (127.58071 34.97252),광양읍
...,...,...,...,...
230,264.0,10.0,POINT (127.60482 35.07921),옥룡면
231,1212.0,60.0,POINT (127.61342 35.07067),옥룡면
232,2381.0,96.0,POINT (127.61177 35.00276),옥룡면
233,2090.0,66.0,POINT (127.67472 35.09239),진상면


In [9]:
np.mean(parkingLot['area']/parkingLot['주차면수'])

41.86715394311621

## 2) 26.광양시_도시계획(교통시설) --> traffic_facility_parkingLot

In [10]:
traffic_facility = gpd.read_file('26.광양시_도시계획(교통시설).geojson')

traffic_facility_parkingLot = traffic_facility[traffic_facility['DGM_NM'].str.contains('주차')]  #교통시설 중 '주차'가 들어간 것만 추출
traffic_facility_parkingLot.rename(columns={"DGM_AR":"area"}, inplace=True)  #면적 의미하는 DGM_AR변수명을 area로 변경
traffic_facility_parkingLot['center_x']=traffic_facility_parkingLot['geometry'].centroid.x  #multipoligon형태여서 중심값 추출
traffic_facility_parkingLot['center_y']=traffic_facility_parkingLot['geometry'].centroid.y
traffic_facility_parkingLot['geometry'] = traffic_facility_parkingLot.apply(lambda row : Point([row['center_x'], row['center_y']]), axis=1)
traffic_facility_parkingLot.drop(['center_x','center_y', 'PRESENT_SN','DGM_LT','DGM_NM'], axis=1, inplace=True)  #필요없는 열들 제거

traffic_facility_parkingLot.reset_index(drop=True, inplace=True)
traffic_facility_parkingLot

,area,geometry
0,344,POINT (127.58425 34.97704)
1,483,POINT (127.58390 34.97837)
2,1612,POINT (127.75138 34.94576)
3,305,POINT (127.70270 34.98894)
4,4608,POINT (127.68964 34.93745)
...,...,...
203,805,POINT (127.58994 34.97387)
204,1244,POINT (127.66659 34.93274)
205,2657,POINT (127.58461 34.97563)
206,1313,POINT (127.69850 34.92671)


## 2-1) 도시계획_주차장 208개 중 행정구역에 속해 있지 않은 4개의 주차장 삭제

In [11]:
#읍면동 넣으려고 했는데 계속 에러나길래 이유가 뭔가 했더니 4개의 주차장이 광양시 행정구역에 안속해 있었음 ㅠㅠ

lst = []
for i in range(len(traffic_facility_parkingLot)):
    for j in range(len(gyboundary)):
        if traffic_facility_parkingLot.geometry[i].within(gyboundary.geometry[j]) == True:
            lst.append(gyboundary.ADM_DR_NM[j])

print("행정구역에 속한 주차장 개수 :", len(lst))

행정구역에 속한 주차장 개수 : 204


In [12]:
lst2 = []
for i in range(len(traffic_facility_parkingLot)):
    for j in range(len(gyboundary)):
        if traffic_facility_parkingLot.geometry[i].within(gyboundary.geometry[j]) == True:
            lst2.append(i)
            
print("행정구역에 속하지 않은 주차장 index :")
for i in range(len(traffic_facility_parkingLot)):
    if i not in lst2:
        print(i)

행정구역에 속하지 않은 주차장 index :
153
154
155
156


In [13]:
#153, 154, 155, 156에 해당하는 주차장 삭제

traffic_facility_parkingLot = traffic_facility_parkingLot.drop([traffic_facility_parkingLot.index[153],traffic_facility_parkingLot.index[154],
                                                               traffic_facility_parkingLot.index[155],traffic_facility_parkingLot.index[156]])
traffic_facility_parkingLot.reset_index(drop=True,inplace=True)
traffic_facility_parkingLot

,area,geometry
0,344,POINT (127.58425 34.97704)
1,483,POINT (127.58390 34.97837)
2,1612,POINT (127.75138 34.94576)
3,305,POINT (127.70270 34.98894)
4,4608,POINT (127.68964 34.93745)
...,...,...
199,805,POINT (127.58994 34.97387)
200,1244,POINT (127.66659 34.93274)
201,2657,POINT (127.58461 34.97563)
202,1313,POINT (127.69850 34.92671)


In [14]:
# 다시 읍면동 넣기
tlist = []
for i in range(len(traffic_facility_parkingLot)):
    for j in range(len(gyboundary)):
        if traffic_facility_parkingLot.geometry[i].within(gyboundary.geometry[j]) == True:
            tlist.append(gyboundary.ADM_DR_NM[j])

traffic_facility_parkingLot['administration'] = tlist
traffic_facility_parkingLot

,area,geometry,administration
0,344,POINT (127.58425 34.97704),광양읍
1,483,POINT (127.58390 34.97837),광양읍
2,1612,POINT (127.75138 34.94576),태인동
3,305,POINT (127.70270 34.98894),옥곡면
4,4608,POINT (127.68964 34.93745),중마동
...,...,...,...
199,805,POINT (127.58994 34.97387),광양읍
200,1244,POINT (127.66659 34.93274),골약동
201,2657,POINT (127.58461 34.97563),광양읍
202,1313,POINT (127.69850 34.92671),중마동


## 2-2) 행정구역에 속하지 않은 주차장 위치 확인

In [15]:
설치현황 = pd.read_csv('01.광양시_충전기설치현황.csv')
lat = 설치현황['lat'].mean()
lon = 설치현황['lon'].mean()


m = folium.Map([lat,lon], zoom_start=11)
folium.GeoJson(gyboundary).add_to(m)

for i in traffic_facility_parkingLot.index:
    sub_lat = traffic_facility_parkingLot.loc[i,"center_y"]
    sub_lon = traffic_facility_parkingLot.loc[i,'center_x']
    title = i
    folium.Marker([sub_lat, sub_lon], tooltip=title).add_to(m)
m

KeyError: 'center_y'

## 3) 두 테이블 합치기 (parkingLot + traffic_facility_parkingLot)

In [16]:
new_parkingLot = pd.concat([parkingLot, traffic_facility_parkingLot], ignore_index=True)
new_parkingLot['category'] = '주차장'

new_parkingLot

,area,주차면수,geometry,administration,category
0,4189.0,161.0,POINT (127.58688 34.97032),광양읍,주차장
1,2968.0,125.0,POINT (127.69543 34.93625),중마동,주차장
2,2836.0,89.0,POINT (127.58459 34.97564),광양읍,주차장
3,1980.0,52.0,POINT (127.58068 34.97013),광양읍,주차장
4,1970.0,64.0,POINT (127.58071 34.97252),광양읍,주차장
...,...,...,...,...,...
434,805.0,NaN,POINT (127.58994 34.97387),광양읍,주차장
435,1244.0,NaN,POINT (127.66659 34.93274),골약동,주차장
436,2657.0,NaN,POINT (127.58461 34.97563),광양읍,주차장
437,1313.0,NaN,POINT (127.69850 34.92671),중마동,주차장


# 야영장

In [17]:
campsite = pd.read_csv('05.광양시_대중집합시설_야영장.csv')
campsite['category'] = '야영장'
campsite['소유주체(공공/민간)'] = np.where(campsite['소유주체(공공/민간)']=='민간',0,1) #민간 = 0, 공공 = 1
campsite['lon'] = campsite['lon'].astype(float)
campsite['lat'] = campsite['lat'].astype(float)
campsite['geometry'] = campsite.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)
campsite = gpd.GeoDataFrame(campsite, geometry='geometry')
campsite = campsite[['category','주소','소유주체(공공/민간)','geometry']]
campsite

,category,주소,소유주체(공공/민간),geometry
0,야영장,광양시 옥룡면 신재로 1678-54,0,POINT (127.60179 35.09657)
1,야영장,광양시 다압면 토끼재길 119-32,0,POINT (127.71085 35.06055)
2,야영장,광양시 옥룡면 약수제단길 154,0,POINT (127.63156 35.06955)
3,야영장,광양시 봉강면 성불로 1097,0,POINT (127.56249 35.07619)
4,야영장,광양시 옥룡면 백계로 337,1,POINT (127.59118 35.05422)
5,야영장,광양시 봉강면 봉당리 178,1,POINT (127.58164 35.00523)


In [18]:
ad = campsite['주소'].tolist()
af = []
for i in range(len(campsite)):
    x = ad[i][4:7]
    af.append(x)
            
campsite['주소'] = af
campsite.rename(columns={"주소":"administration"}, inplace=True)
campsite

,category,administration,소유주체(공공/민간),geometry
0,야영장,옥룡면,0,POINT (127.60179 35.09657)
1,야영장,다압면,0,POINT (127.71085 35.06055)
2,야영장,옥룡면,0,POINT (127.63156 35.06955)
3,야영장,봉강면,0,POINT (127.56249 35.07619)
4,야영장,옥룡면,1,POINT (127.59118 35.05422)
5,야영장,봉강면,1,POINT (127.58164 35.00523)


# 대중집합시설_생활체육시설

In [19]:
생체 = pd.read_csv('04.광양시_대중집합시설_생활체육시설.csv')
생체

,업종,상호,주소,lon,lat
0,가상체험 체육시설업,리얼야구존,"전라남도 광양시 광장로 142, 5~6층 (중동)",127.695281,34.936666
1,가상체험 체육시설업,레전드스크린야구존,"전라남도 광양시 오류2길 2, 3층 (중동)",127.696366,34.936245
2,가상체험 체육시설업,스크린골프존파크 옥곡 공단점,전라남도 광양시 옥곡면 신금산단5길 41,127.716192,34.977066
3,골프연습장,체인지 골프 아카데미,"광양시 진등1길 3-8, 1층 (중동)",127.688989,34.950379
4,골프연습장,패밀리실내골프연습장업,광양시 진등길 6 (중동),127.690767,34.949221
...,...,...,...,...,...
182,체육도장업,용강체육관,"광양시 용강1길 11, 창덕아파트 1차상가 3층",127.609874,34.973490
183,체육도장업,선교체육관,"광양시 폭포사랑길 99, 백운쇼핑센터 1층 (금호동)",127.732105,34.940196
184,체육도장업,대국체육관,"전라남도 광양시 광양읍 와룡길 9-6, 1층",127.609940,34.971263
185,체육도장업,도솔합기도,"전라남도 광양시 광양읍 희양현로 40, 광주은행 3층",127.587397,34.974488


In [20]:
lifePys = pd.read_csv('04.광양시_대중집합시설_생활체육시설.csv')
lifePys['category'] = '생활체육시설'
lifePys.rename(columns={"주소":"administration"}, inplace=True)
lifePys['lon'] = lifePys['lon'].astype(float)
lifePys['lat'] = lifePys['lat'].astype(float)
lifePys['geometry'] = lifePys.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)
lifePys = gpd.GeoDataFrame(lifePys, geometry='geometry')
lifePys = lifePys[['category','geometry', 'administration']]
lifePys

,category,geometry,administration
0,생활체육시설,POINT (127.69528 34.93667),"전라남도 광양시 광장로 142, 5~6층 (중동)"
1,생활체육시설,POINT (127.69637 34.93624),"전라남도 광양시 오류2길 2, 3층 (중동)"
2,생활체육시설,POINT (127.71619 34.97707),전라남도 광양시 옥곡면 신금산단5길 41
3,생활체육시설,POINT (127.68899 34.95038),"광양시 진등1길 3-8, 1층 (중동)"
4,생활체육시설,POINT (127.69077 34.94922),광양시 진등길 6 (중동)
...,...,...,...
182,생활체육시설,POINT (127.60987 34.97349),"광양시 용강1길 11, 창덕아파트 1차상가 3층"
183,생활체육시설,POINT (127.73211 34.94020),"광양시 폭포사랑길 99, 백운쇼핑센터 1층 (금호동)"
184,생활체육시설,POINT (127.60994 34.97126),"전라남도 광양시 광양읍 와룡길 9-6, 1층"
185,생활체육시설,POINT (127.58740 34.97449),"전라남도 광양시 광양읍 희양현로 40, 광주은행 3층"


In [21]:
lst = []
for i in range(len(lifePys)):
    for j in range(len(gyboundary)):
        if lifePys.geometry[i].within(gyboundary.geometry[j]) == True:
            lst.append(gyboundary.ADM_DR_NM[j])



lst2 = []
for i in range(len(lifePys)):
    for j in range(len(gyboundary)):
        if lifePys.geometry[i].within(gyboundary.geometry[j]) == True:
            lst2.append(i)
            
print("행정구역에 속하지 않은 생체시설 index :")
for i in range(len(lifePys)):
    if i not in lst2:
        print(i)

행정구역에 속하지 않은 생체시설 index :
148


In [22]:
lifePys = lifePys.drop([lifePys.index[148]])
lifePys.reset_index(drop=True,inplace=True)
lifePys

,category,geometry,administration
0,생활체육시설,POINT (127.69528 34.93667),"전라남도 광양시 광장로 142, 5~6층 (중동)"
1,생활체육시설,POINT (127.69637 34.93624),"전라남도 광양시 오류2길 2, 3층 (중동)"
2,생활체육시설,POINT (127.71619 34.97707),전라남도 광양시 옥곡면 신금산단5길 41
3,생활체육시설,POINT (127.68899 34.95038),"광양시 진등1길 3-8, 1층 (중동)"
4,생활체육시설,POINT (127.69077 34.94922),광양시 진등길 6 (중동)
...,...,...,...
181,생활체육시설,POINT (127.60987 34.97349),"광양시 용강1길 11, 창덕아파트 1차상가 3층"
182,생활체육시설,POINT (127.73211 34.94020),"광양시 폭포사랑길 99, 백운쇼핑센터 1층 (금호동)"
183,생활체육시설,POINT (127.60994 34.97126),"전라남도 광양시 광양읍 와룡길 9-6, 1층"
184,생활체육시설,POINT (127.58740 34.97449),"전라남도 광양시 광양읍 희양현로 40, 광주은행 3층"


In [23]:
tlist = []
for i in range(len(lifePys)):
    for j in range(len(gyboundary)):
        if lifePys.geometry[i].within(gyboundary.geometry[j]) == True:
            tlist.append(gyboundary.ADM_DR_NM[j])

lifePys['administration'] = tlist
lifePys

,category,geometry,administration
0,생활체육시설,POINT (127.69528 34.93667),중마동
1,생활체육시설,POINT (127.69637 34.93624),중마동
2,생활체육시설,POINT (127.71619 34.97707),옥곡면
3,생활체육시설,POINT (127.68899 34.95038),중마동
4,생활체육시설,POINT (127.69077 34.94922),중마동
...,...,...,...
181,생활체육시설,POINT (127.60987 34.97349),광양읍
182,생활체육시설,POINT (127.73211 34.94020),금호동
183,생활체육시설,POINT (127.60994 34.97126),광양읍
184,생활체육시설,POINT (127.58740 34.97449),광양읍


# 도시계획_공공문화체육시설

In [24]:
CulPys = gpd.read_file('25.광양시_도시계획(공공문화체육시설).geojson')
CulPys.rename(columns={"DGM_AR":"area"}, inplace=True)  
CulPys['center_x']= CulPys['geometry'].centroid.x 
CulPys['center_y']= CulPys['geometry'].centroid.y
CulPys['geometry'] = CulPys.apply(lambda row : Point([row['center_x'], row['center_y']]), axis=1)
CulPys.drop(['center_x','center_y', 'PRESENT_SN','DGM_LT','DGM_NM'], axis=1, inplace=True)  #필요없는 열들 제거

CulPys.reset_index(drop=True, inplace=True)
CulPys['category'] ='공공문화체육시설'

In [25]:
tlist = []
for i in range(len(CulPys)):
    for j in range(len(gyboundary)):
        if CulPys.geometry[i].within(gyboundary.geometry[j]) == True:
            tlist.append(gyboundary.ADM_DR_NM[j])

CulPys['administration'] = tlist
CulPys

,area,geometry,category,administration
0,22707,POINT (127.69595 34.94043),공공문화체육시설,중마동
1,17094,POINT (127.69198 34.93378),공공문화체육시설,중마동
2,19265,POINT (127.69038 34.94687),공공문화체육시설,중마동
3,12998,POINT (127.68944 34.95564),공공문화체육시설,중마동
4,20596,POINT (127.68568 34.94758),공공문화체육시설,중마동
...,...,...,...,...
93,810,POINT (127.60734 34.97410),공공문화체육시설,광양읍
94,20383,POINT (127.69960 34.98775),공공문화체육시설,옥곡면
95,37090,POINT (127.72050 34.93458),공공문화체육시설,금호동
96,18208,POINT (127.71875 34.95972),공공문화체육시설,광영동


# 현재 설치되어있는 충전기

In [26]:
chargingStation = pd.read_csv('01.광양시_충전기설치현황.csv')
chargingStation

,충전소명,충전소위치,충전기 운영기관,급속/완속,충전기용량,이용대수,충전기타입,요금정보(원/kw),lon,lat
0,LF스퀘어 광양점,전남 광양시 광양읍 순광로 466 1층,환경부(한국자동차환경협회),급속,200KW 동시충전,2,DC콤보,255.70,127.568300,34.963002
1,광양만권경제자유구역청,전남 광양시 광양읍 인덕로 1100(광양시 제2청사),환경부(한국자동차환경협회),급속,50KW,1,DC차데모/AC3상/DC콤보,255.70,127.583323,34.980334
2,광양송보파인빌5차아파트,전남 광양시 진등길 55-5 (마동 송보파인빌류 아파트) 501동~506동 상가동,케이티,완속,7kw/h,5,AC완속,170.00,127.689013,34.953759
3,광양시 모리스모텔,전남 광양시 광영로 118-1 지상주차장,차지비,완속,7kw/h,2,AC완속,250.00,127.716708,34.964032
4,광양시 백운쇼핑센타,전남 광양시 폭포사랑길 99 주차장,차지비,완속,7kw/h,2,AC완속,245.00,127.732440,34.940444
5,광양시 블루핸즈(광양정비센터),전남 광양시 광양읍 해광로 888 실외주차장,차지비,완속,7kw/h,1,AC완속,245.00,127.582443,34.963334
6,광양시 비치모텔,전남 광양시 진월면 백운1로 389 주차장,차지비,완속,7kw/h,2,AC완속,250.00,127.750981,34.962506
7,광양시 포스코 ICT 광양사업소,전남 광양시 태인4길 20 주차장,차지비,완속,7kw/h,1,AC완속,250.00,127.760072,34.941198
8,광양시 프라자모텔,전남 광양시 광영로 34,차지비,완속,7kw/h,2,AC완속,250.00,127.725127,34.962147
9,광양시 광양읍사무소,전남 광양시 광양읍 남등길 6,한국전기차충전서비스,완속,7kw/h,1,AC완속,255.70,127.580612,34.973057


In [27]:
chargingStation['geometry'] = chargingStation.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)
station_list = []
for i in range(len(chargingStation)):
    for j in range(len(gyboundary)):
        if chargingStation.geometry[i].within(gyboundary.geometry[j]) == True:
            station_list.append(gyboundary.ADM_DR_NM[j])
chargingStation['administration'] = station_list

chargingStation['category'] ='충전기'
chargingStation['급속/완속'] = np.where(chargingStation['급속/완속']=='급속',0,1)  #급속=0, 완속=1

chargingStation.drop(['충전소명','충전소위치','충전기 운영기관','충전기용량','충전기타입','요금정보(원/kw)','이용대수'], axis=1, inplace=True)  #필요없는 열들 제거
chargingStation.reset_index(drop=True, inplace=True)

chargingStation

,급속/완속,lon,lat,geometry,administration,category
0,0,127.568300,34.963002,POINT (127.5683001 34.96300226),광양읍,충전기
1,0,127.583323,34.980334,POINT (127.5833225 34.98033434),광양읍,충전기
2,1,127.689013,34.953759,POINT (127.6890131 34.95375878),중마동,충전기
3,1,127.716708,34.964032,POINT (127.716708 34.96403159),광영동,충전기
4,1,127.732440,34.940444,POINT (127.7324402 34.94044431),금호동,충전기
5,1,127.582443,34.963334,POINT (127.5824426 34.96333405),광양읍,충전기
6,1,127.750981,34.962506,POINT (127.7509811 34.9625059),진월면,충전기
7,1,127.760072,34.941198,POINT (127.7600717 34.94119758),태인동,충전기
8,1,127.725127,34.962147,POINT (127.7251273 34.9621473),광영동,충전기
9,1,127.580612,34.973057,POINT (127.5806116 34.97305714),광양읍,충전기


# 새로운 테이블

In [28]:
new_table = pd.concat([new_parkingLot, campsite, lifePys, CulPys,chargingStation], ignore_index=True)
new_table = new_table[['category', 'area', '주차면수', 'administration', '소유주체(공공/민간)','급속/완속','geometry']]
new_table['lat']=new_table['geometry'].x.tolist()
new_table['lon']=new_table['geometry'].y.tolist()
new_table

,category,area,주차면수,administration,소유주체(공공/민간),급속/완속,geometry,lat,lon
0,주차장,4189.0,161.0,광양읍,NaN,NaN,POINT (127.58688 34.97032),127.586883,34.970324
1,주차장,2968.0,125.0,중마동,NaN,NaN,POINT (127.69543 34.93625),127.695428,34.936252
2,주차장,2836.0,89.0,광양읍,NaN,NaN,POINT (127.58459 34.97564),127.584586,34.975637
3,주차장,1980.0,52.0,광양읍,NaN,NaN,POINT (127.58068 34.97013),127.580683,34.970130
4,주차장,1970.0,64.0,광양읍,NaN,NaN,POINT (127.58071 34.97252),127.580707,34.972523
...,...,...,...,...,...,...,...,...,...
768,충전기,NaN,NaN,광양읍,NaN,0.0,POINT (127.61270 34.97009),127.612698,34.970095
769,충전기,NaN,NaN,광양읍,NaN,1.0,POINT (127.61270 34.97009),127.612698,34.970095
770,충전기,NaN,NaN,광양읍,NaN,0.0,POINT (127.61046 34.96840),127.610457,34.968399
771,충전기,NaN,NaN,광양읍,NaN,1.0,POINT (127.61046 34.96840),127.610457,34.968399


# 소유지 정보 추가

In [33]:
owner_land = gpd.read_file('14.광양시_소유지정보.geojson')
owner_land

,고유번호,법정동코드,법정동명,지번,소유구분코드,소유구분명,국가기관구분코드,국가기관구분,지목코드,지목,토지면적,geometry
0,4623010200201000100,4623010200,전라남도 광양시 황길동,100-100,None,None,None,None,None,None,0.0,"MULTIPOLYGON (((127.64563 34.92095, 127.64558 ..."
1,4623032024103610006,4623032024,전라남도 광양시 옥룡면 동곡리,361-6,None,None,None,None,None,None,0.0,"MULTIPOLYGON (((127.61621 35.09185, 127.61632 ..."
2,4623032024201080000,4623032024,전라남도 광양시 옥룡면 동곡리,108,None,None,None,None,05,임야,240113.0,"MULTIPOLYGON (((127.61412 35.08394, 127.61406 ..."
3,4623033022105110000,4623033022,전라남도 광양시 옥곡면 대죽리,511,01,개인,ZZ,구분없음,02,답,747.0,"MULTIPOLYGON (((127.67042 34.99886, 127.67048 ..."
4,4623033022105110001,4623033022,전라남도 광양시 옥곡면 대죽리,511-1,None,None,None,None,None,None,0.0,"MULTIPOLYGON (((127.67111 34.99869, 127.67126 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
198162,4623034023201970004,4623034023,전라남도 광양시 진상면 비평리,197-4,01,개인,ZZ,구분없음,05,임야,13637.0,"MULTIPOLYGON (((127.70324 35.04567, 127.70231 ..."
198163,4623034023201970011,4623034023,전라남도 광양시 진상면 비평리,197-11,None,None,None,None,None,None,0.0,"MULTIPOLYGON (((127.70405 35.04689, 127.70428 ..."
198164,4623034023201970006,4623034023,전라남도 광양시 진상면 비평리,197-6,01,개인,ZZ,구분없음,05,임야,45093.0,"MULTIPOLYGON (((127.70198 35.04717, 127.70218 ..."
198165,4623034023201970009,4623034023,전라남도 광양시 진상면 비평리,197-9,None,None,None,None,None,None,0.0,"MULTIPOLYGON (((127.70216 35.04498, 127.70209 ..."


In [38]:
zz = owner_land[owner_land['국가기관구분코드']=='ZZ']

In [41]:
owner_land['소유구분명']

0         None
1         None
2         None
3           개인
4         None
          ... 
198162      개인
198163    None
198164      개인
198165    None
198166    None
Name: 소유구분명, Length: 198167, dtype: object

In [ ]:
owner_list=[]
for i in range(len(new_table)):
    for j in range(len(owner_land)):
        if new_table.geometry[i].within(owner_land.geometry[j]) == True:
            owner_list.append(owner_land.소유구분명[j])
        else:
            owner_list.append(None)

# 인구분포

In [6]:
# 60km/h로 15분 --> 15km
거리 = 60*1000/4
거리

15000.0

In [4]:
축척 = sum([
    *(인구분포도['geometry'].bounds['maxx'] - 인구분포도['geometry'].bounds['minx']),
    *(인구분포도['geometry'].bounds['maxy'] - 인구분포도['geometry'].bounds['miny']),
]) / (2 * len(인구분포도)) / 100
축척

1.0004932768754573e-05

In [32]:
인구분포도['geometry'].centroid.x

0        127.525774
1        127.525774
2        127.525774
3        127.525775
4        127.525775
            ...    
47027    127.786620
47028    127.786623
47029    127.786626
47030    127.786629
47031    127.786632
Length: 47032, dtype: float64

In [3]:
인구분포도 = gpd.read_file('08.광양시_격자별인구현황(100X100).geojson')
인구분포도.loc[인구분포도['val'].apply(pd.isna), 'val'] = 0
인구분포도_경위도 = np.stack([인구분포도['geometry'].centroid.x, 인구분포도['geometry'].centroid.y], axis=1).reshape([1, -1, 2]) #(1,47032,2)

In [30]:
batch_size=32
for i in tqdm(range(int(np.ceil(len(new_table)) / batch_size))): #range(0,6)
        s, e = batch_size * i, batch_size * (i+1)
        테이블_경위도 = np.stack([new_table['geometry'].x[s:e], new_table['geometry'].y[s:e]], axis=1).reshape([-1, 1, 2])
        인구분포_조건 = np.sum((테이블_경위도 - 인구분포도_경위도)**2, axis=2)**0.5 < 거리 * 축척  #shape(32,27350)
        인구분포 = np.array([인구분포도.loc[x, 'val'].mean() for x in 인구분포_조건])   #shape(32,)

In [31]:
인구분포

array([3.94010059, 4.03072882, 4.09215835, 4.09309705, 3.56636364,
       4.10421232, 4.06164557, 4.18886185, 4.11463436, 4.05412107,
       4.04064738, 3.96358559, 4.18519484, 4.05768534, 3.7108969 ,
       3.7108969 , 3.63353363, 3.63353363, 4.07288864, 4.12461434,
       4.12461434, 4.35375053, 4.32416891, 4.26915915, 4.46139546,
       3.99641122, 4.27896233, 4.27896233, 4.37441247, 4.34891425,
       4.3175405 , 3.93060682])

# 으앙